In [7]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML, clear_output
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from lxml import html
import hashlib
import webbrowser
import pickle
import codecs
from datetime import datetime
import time
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from dataclasses import dataclass
from enum import Enum
import configparser
import os
import logging

codecs.register_error("strict", codecs.ignore_errors)
DEBUG_LEVEL = logging.INFO
log = logging.getLogger(__name__)

@dataclass
class Monitor():
    name: str = 'monitor'
        
    class DownloadMode(Enum):
        DOWNLOAD_ONLY = -1
        DOWNLOAD_AND_ANALIZE = 0
        ANALIZE_ONLY = 1
        
    class AnalizeMode(Enum):
        FROM_FIRST = -1
        NONE = 0
        FROM_LAST = 1
        
    def __init__(self, name):
        self.name = name
        logging.basicConfig(handlers=[logging.FileHandler(f'{self.name}.log', 'w', 'utf-8')], level=DEBUG_LEVEL, format='%(asctime)s - %(levelname)s: %(module)s, %(lineno)s, %(funcName)s - %(message)s', datefmt='%d.%m.%y %H:%M:%S')
        config = configparser.ConfigParser()
        config.read(f'{name}.ini')
        self.url = config['COMMON']['url']
        self.script = config['COMMON']['script'].replace('|', '')
        self.update_interval = int(config['COMMON']['update_interval'])
        self.download_mode = eval(config['COMMON']['download_mode'])
        self.analize_mode = eval(config['COMMON']['analize_mode'])
        self.article = config._sections['ARTICLE']
        self.comments = config._sections['COMMENTS']
        
    def download(self, url):
        options = Options()
        options.add_argument('--headless')
        service = Service(r'C:\Tools\GeckoDriver\geckodriver.exe')
        service.start()
        driver = webdriver.Remote(service.service_url, options=options)
        content = ''
        driver.get(url)
        ldict = {'url': url, 'self': self, 'driver': driver, 'time': time, 'content': content}
        exec(self.script, globals(), ldict)
        content = ldict['content']
        driver.quit()
        with codecs.open(f'{self.name}.html', 'w', 'utf-8') as f:
            f.write(content)
        log.debug(f'downloaded: {int(len(content)/1024)} KB')
        return content

    def analize(self, content, pattern, index = -1, default = '', ref = False):
        try:
            result = html.fromstring(content).xpath(pattern)[index] if index >= 0 else html.fromstring(content).xpath(pattern)
            result = result.get('href') if ref else result
        except:
            result = default
        return result
    
    def highlight(self, s, df, df_prev):
        df_crcs = set([row.loc['CRC'] for index, row in df.iterrows()])
        df_prev_crcs = set(row.loc['CRC'] for index, row in df_prev.iterrows())
        df_news_crcs = df_crcs - df_prev_crcs
        cells_marked = pd.Series(data='', index=s.index)
        if s.loc['CRC'] in df_news_crcs:
#             print('Новый текст комментария (новый комментарий или изменен существующий): ' + s.loc['Текст'] + ' ЖЕЛТЫЙ')            
#             cells_marked['Текст'] = 'yellow'
            cells_marked = pd.Series(data='yellow', index=s.index)
        else:
            df_prev_item = df_prev.loc[df_prev['CRC'] == s.loc['CRC']]
            df_prev_item_score = df_prev_item.iloc[0]['Оценка']
            df_item_score = s.loc['Оценка']
            if df_item_score != df_prev_item_score:
                if df_item_score > df_prev_item_score:
#                     print('Оценка изменена (плюс): ' + str(df_item_score) + ' ЗЕЛЕНЫЙ')
#                     cells_marked['Оценка'] = 'lightgreen'
                    cells_marked = pd.Series(data='lightgreen', index=s.index)
                else:
#                     print('Оценка изменена (минус): ' + str(df_item_score) + ' КРАСНЫЙ')
#                     cells_marked['Оценка'] = 'red'
                    cells_marked = pd.Series(data='red', index=s.index)
        result = [f'background: {cell}' if cell != '' else '' for cell in cells_marked]
        return result

    def run(self):
        url = self.url
        download_mode = self.download_mode
        analize_mode = self.analize_mode
        while (True):
            content = ''
            if download_mode == self.DownloadMode.DOWNLOAD_AND_ANALIZE or download_mode == self.DownloadMode.DOWNLOAD_ONLY:
                log.info(f'download page {url}')
                page_content = self.download(url)
                if download_mode == self.DownloadMode.DOWNLOAD_ONLY:
                    return None
            log.info(f'analize page {url}')
            r = ''
            with codecs.open(f'{self.name}.html', 'r', 'utf-8') as f:
                r = f.read()
            link = title = text = date = author = score = comments = ''    
            expression = ';'.join([f'{k}={v}' for k, v in self.article.items()])
            ldict = {'url': url, 'self': self, 'r': r, 'link': link, 'title': title, 'text' : text, 'date' : date, 'author' : author, 'score' : score, 'comments' : comments}
            exec(expression, globals(), ldict)
            link = ldict['link']; title = ldict['title']; text = ldict['text']; date = ldict['date']; author = ldict['author']; link = ldict['link']; score = ldict['score']; comments = ldict['comments']
            comments_total = len(comments)
            df = pd.DataFrame(columns=['Оценка', 'Текст', 'Дата', 'Автор', 'CRC'], index=range(len(comments)))
            for i in range(comments_total):
                r = html.tostring(comments[i])
                score = text = author = date = link = md5 = ''
                expression = ';'.join([f'{k}={v}' for k, v in self.comments.items()])
                ldict = {'url': url, 'self': self, 'r': r, 'score': score, 'text' : text, 'author' : author, 'date' : date, 'link': link, 'md5' : md5}
                exec(expression, globals(), ldict)
                score = ldict['score']; text = ldict['text']; author = ldict['author']; date = ldict['date']; link = ldict['link']; md5 = ldict['md5']
                df.loc[i, 'Оценка'] = score; df.loc[i, 'Текст'] = text; df.loc[i, 'Дата'] = f'<a target="_blank" href="{link}">{date}</a>'; df.loc[i, 'Автор'] = author; df.loc[i, 'CRC'] = md5
                i += 1
            df['Оценка'] = pd.to_numeric(df['Оценка'])
            df = df.sort_values(by=['Оценка'], ascending=False)
            df = df[(df['Оценка'] > df['Оценка'].mean())]
            file = f'{self.name}_{datetime.now().strftime("%Y%m%d%H%M%S")}.bin'
            log.info(f'current file:  {file}')
            df.to_pickle(file)
            files = sorted([file for file in os.listdir() if file.endswith('.bin')])
            if len(files) == 0:
                log.error('no files for analize')
                return
            if analize_mode == self.AnalizeMode.FROM_FIRST:
                file_prev = files[0]
            if analize_mode == self.AnalizeMode.FROM_LAST:
                if len(files) > 1:
                    file_prev = files[-2]
                else:
                    file_prev = files[0]
            log.info(f'previous file: {file_prev}')
            if os.path.isfile(file_prev):
                df_prev = pd.read_pickle(file_prev)
#             # DEMO CHANGES INJECTION
#             df_prev.at[2, 'Оценка'] = 2
#             df_prev.at[3, 'Текст'] = 'НОВЫЙ ТЕКСТ'
#             df_prev.at[3, 'CRC'] = hashlib.sha256(f'НОВЫЙ ТЕКСТ'.encode('utf-8')).hexdigest()
#             display(pd.concat([df,df_prev]).drop_duplicates(keep=False))
#             set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
            result = df.style.apply(self.highlight, axis=1, df=df, df_prev=df_prev).hide_columns(subset=(['CRC'])).hide_index().set_properties(**{'text-align': 'left', 'border-color': 'black','border-style' :'solid' ,'border-width': '1px','border-collapse':'collapse'})
            display(result)
            if self.update_interval == 0:
                return

            time.sleep(self.update_interval)
            clear_output(True)

monitor = Monitor('monitor')
monitor.run()